In [31]:
import statsmodels.api as sm
import pandas as pd
import numpy as np

In [32]:
import warnings
warnings.filterwarnings('ignore')

In [33]:
pca = 'pca_'

df = pd.read_csv('datasets/train_set_pca.csv')
test_df = pd.read_csv('datasets/test_set_pca.csv')

In [34]:

if pca == 'pca_':
    X = df.loc[:, 'pca_0':]
else:
    X = df.loc[:, 'month':]
    y = df['cpi_pct']


X_test = test_df.loc[:, X.columns]

In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [36]:
# first we gonna build 2 benchark Models
# the first is just gonna predict the mean of the test target variable
# the second is just gonna predict the previous month cpi

In [37]:
pci_mean = y.mean()

y_test_pred = pd.DataFrame(pci_mean, columns=['pred'], index=range(X_test.shape[0]))
# y_test_pred.to_csv('predictions/benchmark1_mean.csv')

for the second model we already saved the prev cpi value as our test prediction:

`test_df[['cpi_lag1']].rename({'cpi_lag1': 'pred'}, axis=1).reset_index(drop=True).to_csv('predictions/benchmark2_prev.csv')`

In [38]:
# Shuffling lead to very high lambda values wich basicly meant moth models were just predicting mean (y intercept)
# X = X.sample(frac=1)

In [39]:
model = LinearRegression()

model.fit(X, y)

pred = model.predict(X_test)

pd.DataFrame(pred, index=X_test.index, columns=['pred']).to_csv(f'predictions/{pca}ols.csv') 

In [40]:
model = Lasso(max_iter=3000)

params = {
    'alpha': np.logspace(-8, -0.5, 100).tolist()
}

cv = GridSearchCV(model, params, cv=10)
cv.fit(X, y)

print(cv.best_estimator_)

pred = cv.predict(X_test)

pd.DataFrame(pred, index=X_test.index, columns=['pred']).to_csv(f'predictions/{pca}lasso_cv.csv')

Lasso(alpha=0.0002477076355991714, max_iter=3000)


In [41]:
model = Ridge()

params = {
    'alpha': np.linspace(1, 10000, 100).tolist()
}

cv = GridSearchCV(model, params, cv=10)
cv.fit(X, y)

print(cv.best_estimator_)

pred = cv.predict(X_test)

pd.DataFrame(pred, index=X_test.index, columns=['pred']).to_csv(f'predictions/{pca}ridge_cv.csv')

Ridge(alpha=2829.0)


In [42]:
# shuffle Train dataset for CV

In [43]:
model = Lasso(max_iter=3000)

df = pd.read_csv('datasets/train_set.csv').sample(frac=1)

X = df.loc[:, 'month':]
y = df['cpi_pct']

params = {
    'alpha': np.logspace(-8, -0.5, 100).tolist()
}

cv = GridSearchCV(model, params, cv=10)
cv.fit(X, y)

print(cv.best_estimator_)

pred = cv.predict(X_test)

pd.DataFrame(pred, index=X_test.index, columns=['pred']).to_csv(f'predictions/{pca}lasso_cvshuffled.csv')

Lasso(alpha=7.305271542664449e-05, max_iter=3000)


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- pca_0
- pca_1
- pca_10
- pca_100
- pca_101
- ...
Feature names seen at fit time, yet now missing:
- c_claims_1m_pct
- c_claims_24ma
- c_claims_3m_pct
- c_claims_3m_pct_6m_lag
- c_claims_9m_pct
- ...
